
# Extraction et prétraitement des données OSM

Cette section montre comment extraire des données OSM avec Overpass sur une emprise donnée, en fractionnant la zone en tuiles raisonnables, en gérant les limitations de l’API et en sauvegardant un résultat propre en GeoPackage.


## Préparation

On charge les bibliothèques usuelles et les paramètres centralisés dans config.py :

In [ ]:
import time
import random
from itertools import cycle
from typing import List, Optional, Tuple, Iterable

import requests
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
import json

# Seul fichier requis : config.py (présent dans ton projet)
from config import CONFIG_PATH, cadre_file, OUTPUT_DIR, epsg, city

print("CONFIG_PATH:", CONFIG_PATH)
print("cadre_file:", cadre_file)
print("OUTPUT_DIR:", OUTPUT_DIR)
print("EPSG cible:", epsg, "| city:", city)

CONFIG_PATH: ..\proxy\data\raw\insee\overpass_config.json
cadre_file: ..\proxy\data\raw\insee\CadreMarseille.shp
OUTPUT_DIR: ..\proxy\data\processed\overpass_results_Marseille
EPSG cible: 2154 | city: Marseille


> À retenir :

   * `CONFIG_PATH` pointe vers un JSON qui liste les tags OSM à extraire.

   * `cadre_file` est l’emprise (shapefile/gpkg) : c’est notre zone d’étude.

   * `OUTPUT_DIR` est où on sauvegarde les .gpkg.

   * `epsg` est la projection cible (ex : 2154).

> Lecture de la configuration: On veut vérifier que le fichier de configuration est correctement chargé et comprendre son contenu.

In [30]:
with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    conf = json.load(f)

print("Clés de config:", list(conf.keys()))
print("max_edge_deg:", conf.get("overpass", {}).get("max_edge_deg", 0.25))

tags = conf.get("tags", [])
print("Nombre de tags:", len(tags))
print("Exemple de tag:", tags[0] if tags else None)

Clés de config: ['tags']
max_edge_deg: 0.25
Nombre de tags: 55
Exemple de tag: {'key': 'highway', 'value': 'bus_stop'}


> Interprétation :

   * `max_edge_deg` = taille max d’une tuile en degrés → plus petit = plus de requêtes mais plus stables.

   * `tags` est une liste de { "key": "...", "value": "..." }. On extraira un fichier par tag.

## Overpass : endpoints & session HTTP 

Overpass impose des règles d’usage (éviter de marteler, s’identifier). On prépare :

  * une liste de miroirs (rotation en cas d’échec),

  * un User-Agent clair,

  * un délai de politesse entre requêtes.

In [31]:
# Overpass endpoints (HTTPS) – rotation en cas de retry
OVERPASS_ENDPOINTS: List[str] = [
    "https://overpass-api.de/api/interpreter",
    "https://lz4.overpass-api.de/api/interpreter",
    "https://z.overpass-api.de/api/interpreter",
    "https://overpass.kumi.systems/api/interpreter",
]

# Délai (s) pour éviter de marteler l'API
POLITE_DELAY = 1.5

# Session persistante
_SESSION: Optional[requests.Session] = None

def _get_session() -> requests.Session:
    global _SESSION
    if _SESSION is None:
        s = requests.Session()
        s.headers.update({
            "User-Agent": "AttractiveCity/1.0 (contact: hazim)",
            "Accept-Encoding": "gzip, deflate",
        })
        _SESSION = s
    return _SESSION

> À retenir (éthique & robustesse) :

  * Ralentir volontairement (POLITE_DELAY).

  * S’identifier (User-Agent).

  * Tourner sur plusieurs endpoints → résilience.

## Emprise & tuilage 

Une bbox trop grande → requêtes lourdes → timeouts / 429.
Le tuilage permet des requêtes plus petites, fiables, rejouables.

In [32]:
def load_bbox_from_shapefile(path) -> Tuple[float, float, float, float]:
    gdf = gpd.read_file(path)
    gdf = gdf.to_crs(epsg=4326)
    tb = gdf.total_bounds  # [minx, miny, maxx, maxy]
    return tb[0], tb[1], tb[2], tb[3]


def tile_bbox(
    bbox: Tuple[float, float, float, float],
    max_edge_deg: float = 0.25,
) -> Iterable[Tuple[float, float, float, float]]:
    """
    Découpe la bbox en une grille de bboxes plus petites,
    dont l'arête max ≤ max_edge_deg (en degrés).
    """
    minx, miny, maxx, maxy = bbox
    width = maxx - minx
    height = maxy - miny

    nx = max(1, int(width // max_edge_deg) + (1 if width % max_edge_deg > 1e-12 else 0))
    ny = max(1, int(height // max_edge_deg) + (1 if height % max_edge_deg > 1e-12 else 0))

    dx = width / nx
    dy = height / ny

    for iy in range(ny):
        for ix in range(nx):
            x0 = minx + ix * dx
            x1 = minx + (ix + 1) * dx
            y0 = miny + iy * dy
            y1 = miny + (iy + 1) * dy
            yield (x0, y0, x1, y1)

> Interprétation :

  * max_edge_deg est ton levier anti-timeout.

  * Tu peux augmenter (plus rapide mais risqué) ou réduire (plus lent mais sûr).

## Construire la requête Overpass 

Overpass prend (south, west, north, east) et permet d’interroger node / way / relation.
out center fournit un point même pour les ways/relations → pratique pour tout stocker en points.

In [33]:
def build_query(bbox: Tuple[float, float, float, float], key: str, value: str, timeout_s: int = 180) -> str:
    # Format Overpass: south,west,north,east => (miny,minx,maxy,maxx)
    minx, miny, maxx, maxy = bbox
    bbox_str = f"{miny},{minx},{maxy},{maxx}"
    # out center -> récupère un point même pour ways/relations
    query = f"""
    [out:json][timeout:{timeout_s}];
    (
      node["{key}"="{value}"]({bbox_str});
      way["{key}"="{value}"]({bbox_str});
      relation["{key}"="{value}"]({bbox_str});
    );
    out center;
    """
    return query

> À retenir :

  * timeout_s peut être augmenté pour les zones denses.

  * Un tag = une requête (répétée pour chaque tuile).

## Fetch robuste

Le monde réel = erreurs réseau, serveurs saturés, limitations de débit.
On gère :

429 → on respecte Retry-After si présent, sinon backoff exponentiel ;

5xx → on attend et on réessaie ;

rotation automatique des miroirs.

In [34]:
def fetch_osm_data(
    query: str,
    *,
    max_tries: int = 7,
    initial_backoff: float = 2.0,
    request_timeout: int = 180,
) -> dict:
    session = _get_session()
    endpoints = cycle(OVERPASS_ENDPOINTS)
    errors = []

    for attempt in range(1, max_tries + 1):
        url = next(endpoints)
        try:
            time.sleep(POLITE_DELAY)
            resp = session.post(url, data={"data": query}, timeout=request_timeout)

            if resp.status_code == 429:
                retry_after = resp.headers.get("Retry-After")
                if retry_after and retry_after.isdigit():
                    delay = float(retry_after)
                else:
                    delay = initial_backoff * (2 ** (attempt - 1)) + random.uniform(0, 1.0)
                time.sleep(delay)
                continue

            if resp.status_code in (500, 502, 503, 504):
                delay = initial_backoff * (2 ** (attempt - 1)) + random.uniform(0, 1.0)
                time.sleep(delay)
                continue

            resp.raise_for_status()
            return resp.json()

        except requests.exceptions.RequestException as e:
            delay = initial_backoff * (2 ** (attempt - 1)) + random.uniform(0, 1.0)
            errors.append(f"{url}: {e}")
            time.sleep(delay)

    raise RuntimeError("Overpass requests failed after retries:\n" + "\n".join(errors))

> Interprétation :

  * Si tu vois des temps d’attente, c’est normal.

  * Si ça échoue malgré tout : réduis max_edge_deg (requêtes plus petites).

## Transformer le JSON en points & sauvegarder

On veut un format homogène (points) pour tous types OSM (nodes, ways, relations).
out center garantit un centre pour ways/relations.

In [21]:
def extract_elements(osm_json: dict) -> List[dict]:
    elements = []
    for elem in osm_json.get("elements", []):
        osm_type = elem.get("type")  # "node" | "way" | "relation"
        osm_id = elem.get("id")

        if "lat" in elem and "lon" in elem:
            geometry = Point(elem["lon"], elem["lat"])
        elif "center" in elem:
            c = elem["center"]
            geometry = Point(c["lon"], c["lat"])
        else:
            continue

        row = {
            "geometry": geometry,
            "osm_type": osm_type,
            "osm_id": osm_id,
        }
        tags = elem.get("tags", {})
        if tags:
            row.update(tags)
        elements.append(row)
    return elements

In [22]:
def save_elements(elements: List[dict], output_path: Path, motive_value: str):
    if not elements:
        print(f"No data for {output_path.name}")
        return

    gdf = gpd.GeoDataFrame(elements, geometry="geometry", crs="EPSG:4326").to_crs(epsg=epsg)
    gdf["motive_id"] = str(motive_value).upper()

    # Dédup par (osm_type, osm_id)
    if {"osm_type", "osm_id"}.issubset(gdf.columns):
        gdf = gdf.drop_duplicates(subset=["osm_type", "osm_id"])

    output_path.parent.mkdir(parents=True, exist_ok=True)
    gdf.to_file(output_path, driver="GPKG")
    print(f"✓ Data saved: {output_path}  ({len(gdf)} features)")

> Interprétation :

  * `motive_id` te permet de filtrer par type d’équipement ensuite.

  * `.to_crs(epsg=epsg)` → tu travailles directement dans ton CRS de référence (ex : 2154).

## Collecte sur toutes les tuiles 

C’est le cœur de la collecte pour un tag donné : on parcourt les tuiles, on déduplique au fur et à mesure.

In [ ]:
def collect_elements_for_tag(
    bbox: Tuple[float, float, float, float],
    key: str,
    value: str,
    max_edge_deg: float = 0.25,
) -> List[dict]:
    seen = set()  # (osm_type, osm_id)
    all_rows: List[dict] = []

    for tile in tile_bbox(bbox, max_edge_deg=max_edge_deg):
        query = build_query(tile, key, value, timeout_s=180)
        osm_json = fetch_osm_data(query)
        rows = extract_elements(osm_json)

        for r in rows:
            ot, oid = r.get("osm_type"), r.get("osm_id")
            if ot is not None and oid is not None:
                key_ = (ot, oid)
                if key_ in seen:
                    continue
                seen.add(key_)
            all_rows.append(r)

    return all_rows

> Interprétation :

  * La déduplication évite d’écrire deux fois le même objet si une entité chevauche deux tuiles.

  * Le retour = liste de dicts prêts à transformer en GeoDataFrame.

## Dry-run : 1 tuile + 1 tag (Markdown)

Avant de lancer le “vrai” téléchargement, on teste :

  * la connexion Overpass,

  * un tag réel,

  * le format de sortie,

  * la chaîne requête → JSON → points → GPKG.

In [ ]:
# Charger bbox
bbox = load_bbox_from_shapefile(cadre_file)

# Lire la config
with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    conf = json.load(f)

max_edge_deg = float(conf.get("overpass", {}).get("max_edge_deg", 0.25))
tags = conf.get("tags", [])
assert tags, "La config ne contient pas de tags."

# 1ère tuile + 1er tag
first_tile = next(tile_bbox(bbox, max_edge_deg=max_edge_deg))
key, value = tags[0]["key"], tags[0]["value"]
print("Tile:", first_tile, "| Tag:", key, "=", value)

# Requête → fetch → extraction
q = build_query(first_tile, key, value, timeout_s=180)
osm_json = fetch_osm_data(q)
rows = extract_elements(osm_json)
print("Éléments récupérés:", len(rows))

# Sauvegarde temporaire pour voir le format
tmp_out = Path(OUTPUT_DIR) / f"__SAMPLE__{key}_{value}_{city}.gpkg"
save_elements(rows, tmp_out, motive_value=value)

# Aperçu
if tmp_out.exists():
    gdf_tmp = gpd.read_file(tmp_out)
    display(gdf_tmp.head())

Tile: (5.124256045405729, 43.13326966157955, 5.307748201187553, 43.2945089992285) | Tag: highway = bus_stop
Éléments récupérés: 0
No data for __SAMPLE__highway_bus_stop_Marseille.gpkg


> Interprétation :

  * Si len(rows) == 0, soit le tag est rare sur ta zone, soit il faut diminuer max_edge_deg.

  * Tu vois la structure des colonnes (tags OSM) et la géométrie.

## Un tag complet (toutes les tuiles)

On passe du test au téléchargement d’un tag pour toute la zone.

In [25]:
# Choisir un tag par index (ex: 0)
i = 0
key, value = tags[i]["key"], tags[i]["value"]
print("Tag choisi:", key, "=", value)

elements = collect_elements_for_tag(bbox, key, value, max_edge_deg=max_edge_deg)
outfile = Path(OUTPUT_DIR) / f"{key}_{value}_{city}.gpkg"
save_elements(elements, outfile, motive_value=value)

Tag choisi: highway = bus_stop
✓ Data saved: ..\proxy\data\processed\overpass_results_Marseille\highway_bus_stop_Marseille.gpkg  (4650 features)


> Interprétation :

  * Observe le nombre de features.

  * Ouvre le `.gpkg` dans QGIS pour un premier regard spatial si besoin.


## Extraction complète (sécurisée)

On ne retélécharge pas ce qui existe déjà → relançable sans surcoût.
Tu peux couper puis relancer → il reprendra où il en est.

In [35]:
def extract_osm_elements():
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    bbox = load_bbox_from_shapefile(cadre_file)

    with open(CONFIG_PATH, "r", encoding="utf-8") as f:
        config = json.load(f)

    max_edge_deg = float(config.get("overpass", {}).get("max_edge_deg", 0.25))

    for tag in config["tags"]:
        key, value = tag["key"], tag["value"]
        fname = f"{key}_{value}_{city}.gpkg"
        output_path = Path(OUTPUT_DIR) / fname

        if output_path.exists():
            print(f"File already exists, skipping download: {output_path}")
            continue

        print(f"Extraction {key}={value}...")
        elements = collect_elements_for_tag(bbox, key, value, max_edge_deg=max_edge_deg)
        save_elements(elements, output_path, motive_value=value)

# Décommente pour lancer la collecte complète :
# extract_osm_elements()

## Vérifier les sorties 

On s’assure que les fichiers sont bien produits, et on jette un œil au contenu.

In [28]:
from glob import glob

files = sorted(glob(str(Path(OUTPUT_DIR) / f"*_{city}.gpkg")))
print(f"{len(files)} fichier(s) généré(s) dans {OUTPUT_DIR}:")
files[:10]

59 fichier(s) généré(s) dans ..\proxy\data\processed\overpass_results_Marseille:


['..\\proxy\\data\\processed\\overpass_results_Marseille\\amenity_bar_Marseille.gpkg',
 '..\\proxy\\data\\processed\\overpass_results_Marseille\\amenity_bicycle_parking_Marseille.gpkg',
 '..\\proxy\\data\\processed\\overpass_results_Marseille\\amenity_bicycle_rental_Marseille.gpkg',
 '..\\proxy\\data\\processed\\overpass_results_Marseille\\amenity_bus_station_Marseille.gpkg',
 '..\\proxy\\data\\processed\\overpass_results_Marseille\\amenity_cafe_Marseille.gpkg',
 '..\\proxy\\data\\processed\\overpass_results_Marseille\\amenity_childcare_Marseille.gpkg',
 '..\\proxy\\data\\processed\\overpass_results_Marseille\\amenity_cinema_Marseille.gpkg',
 '..\\proxy\\data\\processed\\overpass_results_Marseille\\amenity_clinic_Marseille.gpkg',
 '..\\proxy\\data\\processed\\overpass_results_Marseille\\amenity_college_Marseille.gpkg',
 '..\\proxy\\data\\processed\\overpass_results_Marseille\\amenity_community_centre_Marseille.gpkg']

> Interprétation :

  * Si certains tags manquent → vérifie la config ou relance extract_osm_elements() (il traitera seulement les manquants).

  * Si un fichier est vide → pas forcément un bug, peut être un tag absent de ta zone.

## Affichage d'un fichier

On peut visualiser un des fichiers extraits pour vérifier le contenu.

In [36]:
# Ouvrir un fichier au hasard et regarder quelques lignes
import random
if files:
    sample = random.choice(files)
    print("Exemple :", sample)
    gdf = gpd.read_file(sample)
    display(gdf.head())
else:
    print("Aucun fichier trouvé.")


Exemple : ..\proxy\data\processed\overpass_results_Marseille\amenity_bicycle_rental_Marseille.gpkg


,osm_type,osm_id,amenity,bicycle_rental,capacity,name,name:oc,network,operator,ref,...,survey:date,covered,operator:type,payment:app,note,brand,brand:wikidata,network:wikidata,motive_id,geometry
0,node,391558891,bicycle_rental,docking_station,20,Prado - Paradis,Pradòt - Paradís,levélo,Inurba Mobility,ce4bhh92rcd13ifvqfh0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BICYCLE_RENTAL,POINT (893833.938 6244075.413)
1,node,391828560,bicycle_rental,docking_station,20,Prado - Gabès,Pradòt - Gàbes,levélo,Inurba Mobility,ce4bhg92rcd13ifvqf90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BICYCLE_RENTAL,POINT (893536.723 6243767.036)
2,node,393138363,bicycle_rental,docking_station,40,Prado - Michelet,Pradòt - Michelet,levélo,Inurba Mobility,ce4bhhp2rcd13ifvqfm0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BICYCLE_RENTAL,POINT (894253.095 6244382.594)
3,node,406255275,bicycle_rental,docking_station,20,Livon - Vieux-Port,Livon - Pòrt Vièlh,levélo,Inurba Mobility,ce4bhbh2rcd13ifvqeqg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BICYCLE_RENTAL,POINT (891666.580 6246547.828)
4,node,619870850,bicycle_rental,docking_station,20,Place aux Huiles,NaN,levélo,Inurba Mobility,ce4bgh92rcd13ifvqb40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BICYCLE_RENTAL,POINT (892545.055 6246689.056)


## Bonnes pratiques & dépannage

Respect de l’API : `User-Agent`, délai, tailles raisonnables, retries → tu es un “bon citoyen”.

**Paramètres clés** :

`max_edge_deg` (plus petit = requêtes plus petites, plus sûres),

`timeout_s` (zones denses),

`POLITE_DELAY` (évite de déranger).

**Erreurs courantes** :

  * 429 = surcharge → normal → attends, réessaie, réduis `max_edge_deg`.

  * Timeout = requêtes trop lourdes → idem.

  * Fichier vide = tag rare → pas un bug.

**Reproductibilité** :

  * `extract_osm_elements()` saute les fichiers déjà présents → tu peux relancer sans tout refaire.

  * GeoPackage en EPSG local → prêt pour des analyses/rendus.

## Conclusion

Dans cette section, nous avons exploré comment extraire efficacement des données d'OpenStreetMap en utilisant l'API Overpass. Nous avons couvert les étapes essentielles, de la préparation des paramètres à la gestion des erreurs courantes, en passant par le tuilage de l'emprise et la sauvegarde des données dans un format utilisable.